In [3]:
import mxnet as mx
# import apollocaffe
# out = mx.symbol.load("googlenet-symbol.json")

In [4]:
pretrained_cnn = mx.nd.load("googlenet-0001.params")


In [5]:
out = mx.symbol.load("googlenet-symbol.json")
arg_shapes, out_shapes, _ = out.infer_shape(data=(1,3,480,640),lstm_hidden_seed=(300,250),lstm_mem_seed=(300,250))
print out_shapes

[(300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 4L, 1L, 1L), (300L, 2L), (300L, 2L), (300L, 2L), (300L, 2L), (300L, 2L), (1L, 1024L, 15L, 20L)]


In [6]:
arg_names = out.list_arguments()
# for name in arg_names:
#     if name == "data":
#         continue
#     key = "arg:" + name
#     if key in pretrained:
#         pretrained[key].copyto(arg_dict[name])
#     else:
#         print ("SKIP arguments %s" % name)
print arg_names

['data', 'conv1_7x7_s2_weight', 'conv1_7x7_s2_bias', 'conv2_3x3_reduce_weight', 'conv2_3x3_reduce_bias', 'conv2_3x3_weight', 'conv2_3x3_bias', 'inception_3a_1x1_weight', 'inception_3a_1x1_bias', 'inception_3a_3x3_reduce_weight', 'inception_3a_3x3_reduce_bias', 'inception_3a_3x3_weight', 'inception_3a_3x3_bias', 'inception_3a_5x5_reduce_weight', 'inception_3a_5x5_reduce_bias', 'inception_3a_5x5_weight', 'inception_3a_5x5_bias', 'inception_3a_pool_proj_weight', 'inception_3a_pool_proj_bias', 'inception_3b_1x1_weight', 'inception_3b_1x1_bias', 'inception_3b_3x3_reduce_weight', 'inception_3b_3x3_reduce_bias', 'inception_3b_3x3_weight', 'inception_3b_3x3_bias', 'inception_3b_5x5_reduce_weight', 'inception_3b_5x5_reduce_bias', 'inception_3b_5x5_weight', 'inception_3b_5x5_bias', 'inception_3b_pool_proj_weight', 'inception_3b_pool_proj_bias', 'inception_4a_1x1_weight', 'inception_4a_1x1_bias', 'inception_4a_3x3_reduce_weight', 'inception_4a_3x3_reduce_bias', 'inception_4a_3x3_weight', 'incepti

In [7]:
arg_dict = dict(zip(arg_names, [mx.nd.zeros(shape,ctx=mx.gpu(0)) for shape in arg_shapes]))


In [8]:
import pandas as pd
lstm_content = pd.read_hdf("lstm.h5","lstm")
param_name = list(lstm_content["param_name"])
param_ndarray = list(lstm_content["param_ndarray"])

param_dict = dict(zip(param_name,param_ndarray))
for k,v in param_dict.items():
    print k,v.shape

ip_conf3_bias (2,)
ip_bbox_unscaled4_weight (4, 250)
ip_bbox_unscaled3_weight (4, 250)
output_gate_weight (250, 1274)
ip_bbox_unscaled2_bias (4,)
ip_conf4_weight (2, 250)
ip_conf3_weight (2, 250)
forget_gate_weight (250, 1274)
ip_bbox_unscaled4_bias (4,)
input_value_weight (250, 1274)
ip_conf0_weight (2, 250)
ip_conf2_weight (2, 250)
ip_conf1_weight (2, 250)
ip_conf4_bias (2,)
ip_conf0_bias (2,)
input_gate_weight (250, 1274)
ip_bbox_unscaled0_weight (4, 250)
ip_bbox_unscaled1_bias (4,)
ip_conf2_bias (2,)
ip_conf1_bias (2,)
ip_bbox_unscaled0_bias (4,)
ip_bbox_unscaled3_bias (4,)
ip_bbox_unscaled2_weight (4, 250)
ip_bbox_unscaled1_weight (4, 250)


In [9]:
for name in arg_names:
    if name in ["data","lstm_mem_seed","lstm_hidden_seed"]:
        continue
    key = "arg:" + name
    if key in pretrained_cnn:
        pretrained_cnn[key].copyto(arg_dict[name])
    elif "input_value" in name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict["input_value_weight"]).copyto(arg_dict[name])
    elif "output_gate" in name:
#         print arg_dict[name].shape
#         print param_dict["output_gate_weight"].shape
        mx.nd.array(param_dict["output_gate_weight"]).copyto(arg_dict[name])
    elif "input_gate" in name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict["input_gate_weight"]).copyto(arg_dict[name])
    elif "forget_value" in name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict["forget_gate_weight"]).copyto(arg_dict[name])
    elif name in param_name:
#         print arg_dict[name].shape
        mx.nd.array(param_dict[name]).copyto(arg_dict[name])
    else:
        print ("SKIP arguments %s" % name)

In [10]:
# model = mx.model.FeedForward(out,num_epoch=100,learning_rate=0.01)
mx.model.save_checkpoint("listen_mxnet_first",epoch=0,symbol=out,arg_params=arg_dict,aux_params={})


In [11]:
# ls = mx.nd.load("listen_mxnet_first-0000.params")
# print ls.keys()

In [12]:
from utils import (annotation_jitter, image_to_h5_mx,
                   annotation_to_h5, load_data_mean_mx)
from utils.annolist import AnnotationLib as al
import time
import os
from scipy.misc import imread,imsave
import random
from utils.annolist import AnnotationLib as al
# from train import load_idl, forward
from utils import (annotation_jitter,
                   annotation_to_h5, Rect, stitch_rects)



def load_idl(idlfile, data_mean, jitter=True):
    """Take the idlfile, data mean and net configuration and create a generator
    that outputs a jittered version of a random image from the annolist
    that is mean corrected."""

    annolist = al.parse(idlfile)
    annos = [x for x in annolist]
    for anno in annos:
        anno.imageName = os.path.join(
            os.path.dirname(os.path.realpath(idlfile)), anno.imageName)
    while True:
        random.shuffle(annos)
        for anno in annos:
            img = imread(anno.imageName)
            im_input = image_to_h5_mx(img, image_mean, image_scaling=1.0)
            yield im_input
            
        break

image_mean = load_data_mean_mx("./data/brainwash_mean.npy", 640,480, image_scaling=1.0)

In [13]:
input_gen = load_idl("./data/brainwash/brainwash_train.idl",image_mean, jitter = False)

In [22]:
from collections import namedtuple
import cv2
max_len = 5

executor = out.bind(ctx=mx.gpu(), args=arg_dict)
e2eExcutor = namedtuple("e2eExcutor",['executor','data','lstm_mem_seed','lstm_hidden_seed','bbox', 'conf', 'arg_dict'])

timeExcutor = e2eExcutor(executor = executor, 
                         data = arg_dict["data"], 
                         lstm_mem_seed = arg_dict["lstm_mem_seed"], 
                         lstm_hidden_seed = arg_dict["lstm_hidden_seed"],
                         bbox = executor.outputs[:max_len],
                         conf = executor.outputs[max_len:2*max_len],
                         arg_dict=arg_dict)


In [23]:
# def overlap_union(x1,y1,x2,y2,x3,y3,x4,y4):
#     SI = max(0, min(x2,x4)-max(x1,x3)) * max(0, min(y2,y4)-max(y1,y3))
#     SU = (x2-x1)*(y2-y1) + (x4-x3)*(y4-y3) - SI + 0.0
#     return SI/SU

def generate_rois(batch_idx,acc_rects):
    proposals =  []
    for rect in acc_rects:
        left_topx = rect.cx - int(rect.width/2)
        left_topy = rect.cy - int(rect.height/2)
        proposals.append([batch_idx,left_topx,left_topy,left_topx+rect.width,left_topy+rect.height])
    return proposals


In [31]:
# grid_width = 20
# grid_height = 15
# img_width = 640
# img_height = 480

start = time.clock()
num = 0

# img_set = input_gen.next()
for i in range(500):
#     im_input = 
    timeExcutor.lstm_mem_seed[:] = mx.nd.zeros((300,250),mx.gpu())
    timeExcutor.lstm_hidden_seed[:] =  mx.nd.zeros((300,250),mx.gpu())
    timeExcutor.data[:] = mx.nd.array(input_gen.next())

    timeExcutor.executor.forward()

    bbox_list = [timeExcutor.bbox[idx].asnumpy() for idx in range(max_len)]
    conf_list = [timeExcutor.conf[idx].asnumpy() for idx in range(max_len)]

#     all_rects = [[[] for x in range(grid_width)] for y in range(grid_height)]
#     generate_rois()
#     pix_per_w = img_width/grid_width
#     pix_per_h = img_height/grid_height
#     for n in range(len(bbox_list)):
#         for k in range(grid_height * grid_width):
#             #print k,n,"k n"
#             y = int(k / grid_width)
#             x = int(k % grid_width)
#             bbox = bbox_list[n][k]
#             conf = conf_list[n][k,1].flatten()[0]
#             abs_cx = pix_per_w/2 + pix_per_w*x + int(bbox[0,0,0])
#             abs_cy = pix_per_h/2 + pix_per_h*y + int(bbox[1,0,0])
#             w = bbox[2,0,0]
#             h = bbox[3,0,0]
#             #print x,y
#             all_rects[y][x].append(Rect(abs_cx,abs_cy,w,h,conf))
#     acc_rects = stitch_rects(all_rects)


# for rect in acc_rects:

#     if rect.true_confidence < 0.6:
#         continue
#     cv2.rectangle(img, (rect.cx-int(rect.width/2), rect.cy-int(rect.height/2)),\
#      (rect.cx+int(rect.width/2), rect.cy+int(rect.height/2)),color=(0,0,255),thickness=2)
# imsave("./mxnet/%i.jpg"% i, img)
print 500/(time.clock() - start)

17.5706502669
